# Rag From Scratch: Overview

### The process of building RAG app(s) from scratch.

## Enviornment

In [ ]:
!pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain

In [1]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.document_loaders import PyPDFLoader

In [2]:
# Loading the PDF file using PyPDFLoader
loder = PyPDFLoader(r'c:\Users\imran\Downloads\BAHRAIN CBB Rulebook Appendix - 2 ESG Reporting Frameworks and Global Initiatives.pdf')
pages = loder.load()
# pages

# Combining all the pages into a single text
document_text = "\n".join([page.page_content for page in pages])
document_text


' Central Bank of Bahrain  \nRulebook  Common Volume  \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \nENVIRONMENTAL, SOCIAL AND \nGOVERNANCE REQUIREMENTS \nMODULE  \n \n \n \n \n \n \n \n \n \n \n Central Bank of Bahrain  \nRulebook  Common Volume   \n \n \n MODULE:  ESG  (Environmental, Social and Governance  Requ irements )  \n                                                Table of Contents  \n \n \n \n Date \nLast \nChanged     \nESG -A Introduction   \n ESG -A.1 Introduction and Scope  XX/2023  \n ESG -A.2 Module History  XX/2023  \n    \nESG -1 Reporting Requirements    \n ESG -1.1 ESG Key Performance Indicators  XX/2023  \n ESG -1.2 ESG Reporting Process  XX/2023  \n    \n    \n    \nAPPENDICES     \nAppendix 1  ESG KPIs and Guidance   \nAppendix 2 ESG Reporting Frameworks and Global Initiatives   \n    \n \n \n \n Central Bank of Bahrain  \nRulebook  Common Volume   \n \n \n MODULE  ESG :  Environmental, Social and Governance  Requirements  \nCHAPTER  ESG -A:   Introduction   \n

In [3]:

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(document_text)
texts

['Central Bank of Bahrain  \nRulebook  Common Volume  \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \nENVIRONMENTAL, SOCIAL AND \nGOVERNANCE REQUIREMENTS \nMODULE  \n \n \n \n \n \n \n \n \n \n \n Central Bank of Bahrain  \nRulebook  Common Volume   \n \n \n MODULE:  ESG  (Environmental, Social and Governance  Requ irements )  \n                                                Table of Contents  \n \n \n \n Date \nLast \nChanged     \nESG -A Introduction   \n ESG -A.1 Introduction and Scope  XX/2023  \n ESG -A.2 Module History  XX/2023  \n    \nESG -1 Reporting Requirements    \n ESG -1.1 ESG Key Performance Indicators  XX/2023  \n ESG -1.2 ESG Reporting Process  XX/2023  \n    \n    \n    \nAPPENDICES     \nAppendix 1  ESG KPIs and Guidance   \nAppendix 2 ESG Reporting Frameworks and Global Initiatives   \n    \n \n \n \n Central Bank of Bahrain  \nRulebook  Common Volume   \n \n \n MODULE  ESG :  Environmental, Social and Governance  Requirements  \nCHAPTER  ESG -A:   Introduction   \n

In [5]:
embeddings = HuggingFaceEmbeddings()
db = Chroma.from_texts(texts, embeddings)
db

ImportError: Could not import sentence_transformers python package. Please install it with `pip install sentence-transformers`.